## LAB 4 Jak działa GPT
#### Predykcja następnego znaku w tekście z wykorzystaniem torch.nn.Embedding

Celem laboratorium jest:

* zapoznanie z modułem torch.nn
* zapoznanie z warstwą osadzeń wektorowych (embedding)

In [1]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

In [2]:
torch.set_printoptions(precision=4, sci_mode=False)

In [3]:
import requests

url = "https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/data/hpmor_part.txt?raw=true"
response = requests.get(url)
text = response.text

In [4]:
chars = sorted(list(set(text)))
n_tokens = len(chars) # liczba znaków
idx_to_ch = {i: c for i, c in enumerate(chars)}
ch_to_idx = {c: i for i, c in enumerate(chars)}

##### Generacja tekstu
* Teraz w funkcji generującej tekst wykorzystujemy model do wyznaczenia logitów.
* Prawdopodobieństwa wyznaczamy z wykorzystamiem F.softmax.

In [ ]:
def generate_text(start_seq, model, max_size):
    '''
    Funkcja generuje tekst.
    start_seq (str) - początek tekstu, podany przez użytkownika
    norm_counts - znormalizowana macierz zliczeń
    max_size (int) - zadana długość tekstu
    '''
    for i in range(max_size):
        last_ch = start_seq[-1]
        logits = model(torch.tensor(ch_to_idx[last_ch], device=device)) #<- zmiana
        probs = F.softmax(logits, dim=0) #<- zmiana
        probs = probs.cpu().detach().numpy()
        next_ch = idx_to_ch[np.random.choice(n_tokens, p=probs)]
        start_seq += next_ch
    return start_seq

In [ ]:
def idx_to_tokens(idx):
    '''
    funkcja zamienia listę pozycji w tekście (o długości równej rozmiar batcha) na tensor o wymiarach 
    (rozmiar batcha x 1) zawierający indeksy tokenów
    '''
    x = [text[i] for i in idx]
    x = [ch_to_idx[xx] for xx in x]
    x = torch.tensor(x)
    return x

##### Dane uczące
* Zamiast kodowania one-hot będziemy stosować **indeksy** tokenów

In [ ]:
def get_batch(batch_size=8):
    '''
    funkcja zwraca batch danych uczących
    x i y to tensory o wymiarach (rozmiar batcha x rozmiar słownika) zawierające indeksy tokenów
    '''
    rand_idx = np.random.randint(0, len(text) - 1, size=batch_size)
    x = idx_to_tokens(rand_idx)
    y = idx_to_tokens(rand_idx + 1)
    return x, y

In [ ]:
x, y = get_batch()

In [ ]:
x

In [ ]:
y

#### Embedding Model
* Zaimplementować klasę EmbeddingModel
* Model ma jedną warstwę typu Embedding o wymiarze liczba tokenów x liczba tokenów

In [ ]:
class EmbeddingModel(nn.Module): 
    # TODO

In [ ]:
model = EmbeddingModel()

In [ ]:
for name, param in model.named_parameters():
    print(name)
    print(param.shape)

In [ ]:
model(x).shape

In [ ]:
# przenosimy model na gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmbeddingModel().to(device)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=5)

In [ ]:
n_steps = 1500
batch_size = 2048

#### Zaimplementować pętlę uczenia
* Należy wzorować się na przykładzie z wykładu
* W przeciwieństwie do wykładu mamy dane w batchach
* Każdy batch po wygenerowaniu trzeba przenieść na device
* Powinno dać się uzyskać stratę na poziomie około 2.5

In [ ]:
# TODO

In [ ]:
print(generate_text("T", model, 200))